# 数据准备

In [6]:
import jsonlines
import itertools
import pandas as pd
from pprint import pprint
import datasets
from datasets import load_dataset
print('done')

done


In [ ]:
pretrained_datasets = load_dataset("opencsg/chinese-fineweb-edu", split = "train", streaming=True)

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /datasets/opencsg/chinese-fineweb-edu/resolve/main/README.md (Caused by ConnectTimeoutError(<HTTPSConnection(host='huggingface.co', port=443) at 0x15cfe3e00>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 38516ad6-c581-4b8d-8954-7117329d0ce3)')' thrown while requesting HEAD https://huggingface.co/datasets/opencsg/chinese-fineweb-edu/resolve/main/README.md
Retrying in 1s [Retry 1/5].
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /datasets/opencsg/chinese-fineweb-edu/resolve/main/README.md (Caused by ConnectTimeoutError(<HTTPSConnection(host='huggingface.co', port=443) at 0x15d052e90>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: b6813a76-0ed2-442d-b0c8-78c67ec616b1)')' thrown while requesting HEAD https://huggingface.co/datasets/opencsg/chinese-fineweb-edu/res

In [ ]:
n = 1
print("Pretrained dataset:")
top_n = itertools.islice(pretrained_datasets,n)
for i in top_n:
    print(i)


In [6]:
instruction_dataset_df = load_dataset("kotzeje/lamini_docs.jsonl", split = "train")
print(instruction_dataset_df)

Dataset({
    features: ['question', 'answer'],
    num_rows: 1400
})


In [7]:
n = 5
print("Pretrained dataset:")
top_n = itertools.islice(instruction_dataset_df,n)
for i in top_n:
    print(i)

Pretrained dataset:
{'question': 'How can I evaluate the performance and quality of the generated text from Lamini models?', 'answer': "There are several metrics that can be used to evaluate the performance and quality of generated text from Lamini models, including perplexity, BLEU score, and human evaluation. Perplexity measures how well the model predicts the next word in a sequence, while BLEU score measures the similarity between the generated text and a reference text. Human evaluation involves having human judges rate the quality of the generated text based on factors such as coherence, fluency, and relevance. It is recommended to use a combination of these metrics for a comprehensive evaluation of the model's performance."}
{'question': "Can I find information about the code's approach to handling long-running tasks and background jobs?", 'answer': 'Yes, the code includes methods for submitting jobs, checking job status, and retrieving job results. It also includes a method for

In [8]:
examples = instruction_dataset_df.to_dict()
text = examples["question"][0] + examples["answer"][0]
text

"How can I evaluate the performance and quality of the generated text from Lamini models?There are several metrics that can be used to evaluate the performance and quality of generated text from Lamini models, including perplexity, BLEU score, and human evaluation. Perplexity measures how well the model predicts the next word in a sequence, while BLEU score measures the similarity between the generated text and a reference text. Human evaluation involves having human judges rate the quality of the generated text based on factors such as coherence, fluency, and relevance. It is recommended to use a combination of these metrics for a comprehensive evaluation of the model's performance."

In [9]:
prompt_template_qa = """### Question:
{question}
### Answer: 
{answer}"""

In [10]:
print(prompt_template_qa)

### Question:
{question}
### Answer: 
{answer}


In [11]:
question = examples["question"][0]
answer = examples["answer"][0]

text_with_prompt_template =  prompt_template_qa.format(question=question, answer = answer)
print(text_with_prompt_template)

### Question:
How can I evaluate the performance and quality of the generated text from Lamini models?
### Answer: 
There are several metrics that can be used to evaluate the performance and quality of generated text from Lamini models, including perplexity, BLEU score, and human evaluation. Perplexity measures how well the model predicts the next word in a sequence, while BLEU score measures the similarity between the generated text and a reference text. Human evaluation involves having human judges rate the quality of the generated text based on factors such as coherence, fluency, and relevance. It is recommended to use a combination of these metrics for a comprehensive evaluation of the model's performance.


In [12]:
prompt_template_q = """### Question:
{question}
### Answer:"""

In [13]:
num_examples = len(examples["question"])
finetuning_dataset_text_only = []
finetuning_dataset_question_answer = []
for i in range(num_examples):
    question = examples["question"][i]
    answer = examples["answer"][i]

    text_with_prompt_template_qa = prompt_template_qa.format(question=question, answer = answer)
    finetuning_dataset_text_only.append({"text": text_with_prompt_template_qa})

    text_with_prompt_template_q = prompt_template_q.format(question=question)
    finetuning_dataset_question_answer.append({"question": text_with_prompt_template_q ,"answer": answer})

In [14]:
pprint(finetuning_dataset_text_only[0])

{'text': '### Question:\n'
         'How can I evaluate the performance and quality of the generated text '
         'from Lamini models?\n'
         '### Answer: \n'
         'There are several metrics that can be used to evaluate the '
         'performance and quality of generated text from Lamini models, '
         'including perplexity, BLEU score, and human evaluation. Perplexity '
         'measures how well the model predicts the next word in a sequence, '
         'while BLEU score measures the similarity between the generated text '
         'and a reference text. Human evaluation involves having human judges '
         'rate the quality of the generated text based on factors such as '
         'coherence, fluency, and relevance. It is recommended to use a '
         'combination of these metrics for a comprehensive evaluation of the '
         "model's performance."}


In [15]:
pprint(finetuning_dataset_question_answer[0])

{'answer': 'There are several metrics that can be used to evaluate the '
           'performance and quality of generated text from Lamini models, '
           'including perplexity, BLEU score, and human evaluation. Perplexity '
           'measures how well the model predicts the next word in a sequence, '
           'while BLEU score measures the similarity between the generated '
           'text and a reference text. Human evaluation involves having human '
           'judges rate the quality of the generated text based on factors '
           'such as coherence, fluency, and relevance. It is recommended to '
           'use a combination of these metrics for a comprehensive evaluation '
           "of the model's performance.",
 'question': '### Question:\n'
             'How can I evaluate the performance and quality of the generated '
             'text from Lamini models?\n'
             '### Answer:'}


In [16]:
with jsonlines.open(f'lamini_doc_processed.jsonl', 'w') as writer:
    writer.write_all(finetuning_dataset_question_answer)

In [17]:
finetuning_dataset_name = "lamini/lamini_docs"
finetuning_dataset = load_dataset(finetuning_dataset_name)
print(finetuning_dataset)

README.md:   0%|          | 0.00/577 [00:00<?, ?B/s]

data/train-00000-of-00001-5cdebbc48da413(…):   0%|          | 0.00/615k [00:00<?, ?B/s]

data/test-00000-of-00001-4c77a066a883f33(…):   0%|          | 0.00/83.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1260 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/140 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 140
    })
})
